# Search Engines Market Analysis

In [ ]:
%matplotlib inline

import sys
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mlp
import matplotlib.pyplot as plt

from itertools import product
from dateutil.parser import parse
from datetime import date, timedelta

In [ ]:
# Plot
plt.style.use('bmh')
mlp.rcParams['figure.figsize'] = (16, 8)

In [ ]:
%%capture
# R
from rpy2.robjects import r, pandas2ri as pd2
pd2.activate()
sys.path.extend(['./forecast-master', './forecast-master/forecast'])
from forecast import forecast, stl, stldf, bind

## Data Reading

Считываем датасет `my_raw_data.csv`, полученный в `fetch.ipynb`.

* `date` - дата по дням с 2014-01-01 до 2017-09-28 включительно
* `se` - поисковая система, либо yandex либо search
* `browser` - браузер, 5 типов
* `hits` - количество хитов

In [ ]:
df = pd.read_csv('my_raw_data.csv')
df.isnull().values.any()  # Check if df has nan values.

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
FIRST_DAY, LAST_DAY = parse(df.date[0]), parse(df.date.iloc[-1])
FIRST_DAY, LAST_DAY

In [ ]:
df.describe()

In [ ]:
SES, BROWSERS = np.unique(df['se']).tolist(), np.unique(df['browser']).tolist()
SES, BROWSERS

## Data Prerocessing

Устраняем аномалии в данных.

### Дополним недостающие данные

Заметим, что дней с первого до последнего рассматриваемого дня:

In [ ]:
(LAST_DAY - FIRST_DAY).days + 1

А в датасета на каждую из 10 пар se, brower есть:

In [ ]:
df.groupby(['se', 'browser']).size()

Пропущенные значение - 4 дня от 2014-12-28 до 2014-12-31, пока восстановим их нулями.  

#NOTE: в новом датасет такой проблемы нет

In [ ]:
# missing_dates = ['2014-12-28', '2014-12-29', '2014-12-30', '2014-12-31']
# for date, se, browser in product(missing_dates, SES, BROWSERS):
#     df = df.append({'date': date, 'se': se, 'browser': browser, 'hits': 0.0}, ignore_index=True)

df.date = df.date.astype('datetime64[ns]')
df.sort_values(df.columns.tolist(), inplace=True)
df.reset_index(drop=True, inplace=True)
df.dtypes

Продолжим разбираться с данными.

In [ ]:
def plot_pivot(df_to_plot, columns, *, title=None):
    data_args = {'index': 'date', 'values': 'hits', 
                 'columns': columns, 'aggfunc': np.sum}
    plot_args = {'title': title or f'{columns} pivot'}
    df_to_plot.pivot_table(**data_args).plot(**plot_args)


def plot_fix(fix, df_to_plot, columns):
    plot_pivot(df_to_plot, columns, title=f'before {fix.__name__}')
    plot_pivot(fix(df_to_plot), columns, title=f'after {fix.__name__}')

### Разрез `se`

In [ ]:
plot_pivot(df, 'se')

Что можно заметить?  
Во-первых, первые пару дней в январе 2014 аномально маленькие (хоть это и новогодние праздники, но в других годах падение не такое существенное). Посмотрим поближе.

In [ ]:
plot_pivot(df[df.date <= '2014-02-01'], 'se')

Видно, что можем отрезать префикм до 9-го числа.

In [ ]:
def fix_prefix(df): return df[df.date >= '2014-01-09']
plot_fix(fix_prefix, df, 'se')

In [ ]:
df = fix_prefix(df)

Теперь посмотрим на аномальное низкие показатели в районе 2014-09, новый год 15-го и в районе 2015-03-15.

In [ ]:
plot_pivot(df[('2014-08-10' <= df.date) & (df.date <= '2014-08-30')], 'se', title='zero day')
plot_pivot(df[('2014-11-25' <= df.date) & (df.date <= '2015-02-10')], 'se', title='zero span')
plot_pivot(df[('2015-03-15' <= df.date) & (df.date <= '2015-04-10')], 'se', title='low anomally')

Зафиксируем промежутки (значение подберем эмпирически - так быстрее).

In [ ]:
ZDAY = '2014-08-21'
ZSPAN = '2014-12-09', '2015-01-29'
LSPAN = '2015-03-27', '2015-04-04'

Напишем фикс.  
Для нулевого дня возьмем среднее ближайших, для нулевого промежутка - перенесем и отскалируем со следующего года.  
Для спайка перенесем со следующей недели.

In [ ]:
def fix_zeros_and_lower(df):
    df = df.copy()
    zday = parse(ZDAY)
    zleft, zright = parse(ZSPAN[0]), parse(ZSPAN[1])
    lleft, lright = parse(LSPAN[0]), parse(LSPAN[1])
    
    def past(d, days=1): return d - timedelta(days=days)
    def seek(d, days=1): return d + timedelta(days=days)
    
    for se, browser in product(SES, BROWSERS):
        cond = (df.se == se) & (df.browser == browser)
        
        prev_day = df.loc[cond & (df.date == past(zday)), 'hits'].values[0]
        next_day = df.loc[cond & (df.date == seek(zday)), 'hits'].values[0]
        df.loc[cond & (df.date == zday), 'hits'] = (prev_day + next_day) // 2
        
        nleft, nright = seek(zleft, days=365), seek(zright, days=365)
        zleft_val = df.loc[cond & (df.date == past(zleft)), 'hits'].values[0]
        nleft_val = df.loc[cond & (df.date == past(nleft)), 'hits'].values[0]
        nspan = df.loc[cond & (nleft <= df.date) & (df.date <= nright), 'hits'].values
        df.loc[cond & (zleft <= df.date) & (df.date <= zright), 'hits'] = nspan - nleft_val + zleft_val
        
        nvalues = df.loc[cond & (seek(lleft, days=7) <= df.date) & (df.date <= seek(lright, days=7)), 'hits']
        df.loc[cond & (lleft <= df.date) & (df.date <= lright), 'hits'] = nvalues.values

    return df


plot_fix(fix_zeros_and_lower, df, 'se')

In [ ]:
old_df = df
df = fix_zeros_and_lower(df)

### Разрез `browser`

In [ ]:
plot_pivot(df, 'browser')

Все ок, кроме спайка у хрома и оперы в серединке между 2014 и 2015. Посмотрим поближе.

In [ ]:
span = ('2014-03-10' <= df.date) & (df.date <= '2015-06-25')
cond = span & ((df.browser == 'opera') | (df.browser == 'chrome'))
plot_pivot(df[cond], 'browser')

Напишем фикс, значения подберем эмпирически.

In [ ]:
SSPAN = '2014-06-15', '2014-06-21'
OSPAN = '2014-06-06', '2014-06-22'


def fix_spikes(df):
    df = df.copy()
    sleft, sright = parse(SSPAN[0]), parse(SSPAN[1])
    oleft, oright = parse(OSPAN[0]), parse(OSPAN[1])
    
    def past(d, days=1): return d - timedelta(days=days)
    def seek(d, days=1): return d + timedelta(days=days)
    
    for se in SES:
        cond = (df.se == se) & (df.browser == 'chrome')
        df.loc[cond & (sleft <= df.date) & (df.date <= sright), 'hits'] *= 0.8
    
    for se in SES:
        cond = (df.se == se) & (df.browser == 'opera')
        df.loc[cond & (oleft <= df.date) & (df.date <= oright), 'hits'] += (1e+7 / 2)
    
    return df


plot_fix(fix_spikes, df, 'browser')

Good. Еще раз посмотрим на оба среза.

In [ ]:
df = fix_spikes(df)

In [ ]:
plot_pivot(df, 'se')
plot_pivot(df, 'browser')

Все ок. Остальные всплески очень похожи на сезонные явления.

In [ ]:
old_df['hits_fixed'] = np.round(df.hits)
df = old_df

## STL

In [ ]:
bind(pd.DataFrame, stldf)
bind(pd.Series, stl, forecast)

new_columns = ['hits_week_trend', 'hits_week_seasonal', 'hits_year_trend', 'hits_year_seasonal']
for column in new_columns:
    df[column] = np.zeros(len(df))

for se, browser in product(SES, BROWSERS):
    cond = (df.se == se) & (df.browser == browser)
    
    ts = pd.Series(df.loc[cond, 'hits_fixed'].tolist(), index=df.loc[cond, 'date'])
    res = ts.stl(frequency=7, method='arima')
    trend = res.iloc[:, 1].as_matrix()
    seasonal = (res.iloc[:, 0] + res.iloc[:, 1]).as_matrix()
    df.loc[cond, 'hits_week_trend'] = np.round(trend)
    df.loc[cond, 'hits_week_seasonal'] = np.round(seasonal)
    
    ts = pd.Series(trend, index=df.loc[cond, 'date'])
    res = ts.stl(frequency=365, method='arima')
    trend = res.iloc[:, 1].as_matrix()
    seasonal = (res.iloc[:, 0] + res.iloc[:, 1]).as_matrix()
    df.loc[cond, 'hits_year_trend'] = np.round(trend)
    df.loc[cond, 'hits_year_seasonal'] = np.round(seasonal)

Посмотрим на снятие недельной сезонности:

In [ ]:
df.pivot_table(index='date', values='hits_week_trend', columns='browser', aggfunc=np.sum).plot();

Годовой:

In [ ]:
df.pivot_table(index='date', values='hits_year_trend', columns='browser', aggfunc=np.sum).plot();

Сохраним результат в csv.

In [ ]:
df.to_csv('clean_data.csv', index=False)